In [1]:
from scipy.io import wavfile
import numpy as np
import IPython.display as ipd
import plotly
import plotly.offline as pyo 
import plotly.graph_objs as go
import matplotlib.pyplot as plt 
import librosa
import soundfile as sf
import pandas as pd 
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from scipy.signal import argrelextrema

## Funciones

In [2]:
#Función para cortar los dataframes, el cambio de ritmo lo vi a ojo, debería hacer algo que detecte automáticamente. 
def corte(p0, pf, df):
  df=df[p0:pf]
  return df


#Función que me detecta los máximos, es decir, el ataque de cada golpe.
def maximos(df, columna, salto, fs, nombre_columna):
  max_indices = argrelextrema(df[columna].values, np.greater_equal, order=salto)[0]
  max_values = df[columna].iloc[max_indices]
  dfmax=pd.DataFrame(max_values)
  dfmax[nombre_columna] = dfmax.index / fs
  dfmax=dfmax.reset_index()
  return dfmax


#Función que me pone 0 en todas las amplitudes menos en los máximos donde pone 1.
def maximos2(df, columna, salto, fs, nombre_columna, columna_nueva):
    
    max_indices = argrelextrema(df[columna].values, np.greater_equal, order=salto)[0]

    df[columna + '_maximos'] = 0
 
    df.loc[df.index[max_indices], columna + '_maximos'] = 1

    dfmax = pd.DataFrame({
        columna + '_maximos': df[columna + '_maximos'],
        nombre_columna: df.index / fs
    })
    df[columna_nueva]=df[columna + '_maximos'].cumsum()

    return df

#Función para calcular la diferencia entre picos
def delta(df, columna):
    delta=pd.DataFrame(df[columna].diff())
    delta=delta.dropna()
    delta=delta.reset_index()
    return delta
#Guardo distancia entre picos metrónomo-tambor, cuando hay más del tambor guardo en "resto" el que no uso, cuando hay de menos
#guardo el que uso dos veces en "repetidos".
def procesar_picos(m, t):
    if len(t) > len(m):
        m['min_diferencia'] = None
        m['tiempo_tambor'] = None

        for i in range(len(m)):
            valor = m.loc[i, 'tiempo_metronomo']

            diferencias = t['tiempo_tambor'] - valor
            closest_index = (diferencias.abs()).idxmin()
            min_dif = diferencias.iloc[closest_index]

            m.loc[i, 'min_diferencia'] = min_dif
            m.loc[i, 'tiempo_tambor'] = t.loc[closest_index, 'tiempo_tambor']
            m.loc[i, 'amplitud_tambor'] = t.loc[closest_index, 'amplitud_tambor']

        resto = t[~t['tiempo_tambor'].isin(m['tiempo_tambor'])].copy()

        resto['distancia_al_mas_cercano'] = resto['tiempo_tambor'].apply(lambda x: abs(x - m['tiempo_metronomo']).min())

        tiempos = pd.concat([m['tiempo_metronomo'], m['tiempo_tambor'], m['min_diferencia'], m['amplitud_tambor']], axis=1)
        return tiempos, resto

    elif len(t) < len(m):
        m['min_diferencia'] = None
        m['tiempo_tambor'] = None
        t['seleccionado'] = 0

        for i in range(len(m)):
            valor = m.loc[i, 'tiempo_metronomo']

            diferencias = t['tiempo_tambor'] - valor
            closest_index = (diferencias.abs()).idxmin()
            min_dif = diferencias.iloc[closest_index]

            m.loc[i, 'min_diferencia'] = min_dif
            m.loc[i, 'tiempo_tambor'] = t.loc[closest_index, 'tiempo_tambor']
            m.loc[i, 'amplitud_tambor'] = t.loc[closest_index, 'amplitud_tambor']

            t.loc[closest_index, 'seleccionado'] += 1

        repetidos = t[t['seleccionado'] > 1].copy()

        repetidos['distancia_al_mas_cercano'] = repetidos['tiempo_tambor'].apply(lambda x: abs(x - m['tiempo_metronomo']).min())

        t.drop(columns=['seleccionado'], inplace=True)

        tiempos = pd.concat([m['tiempo_metronomo'], m['tiempo_tambor'], m['min_diferencia'], m['amplitud_tambor']], axis=1)
        return tiempos, repetidos

    else:
        m['min_diferencia'] = None
        m['tiempo_tambor'] = None

        for i in range(len(m)):
            valor = m.loc[i, 'tiempo_metronomo']

            diferencias = t['tiempo_tambor'] - valor
            closest_index = (diferencias.abs()).idxmin()
            min_dif = diferencias.iloc[closest_index]

            m.loc[i, 'min_diferencia'] = min_dif
            m.loc[i, 'tiempo_tambor'] = t.loc[closest_index, 'tiempo_tambor']
            m.loc[i, 'amplitud_tambor'] = t.loc[closest_index, 'amplitud_tambor']

        tiempos = pd.concat([m['tiempo_metronomo'], m['tiempo_tambor'], m['min_diferencia'], m['amplitud_tambor']], axis=1)
        return tiempos, None
#Función que agrega columna con suma acumulativa de otra.
def indice_sumatoria(df, columna, columna_nueva):
    df[columna_nueva]=df[columna].cumsum()
    return df
#Función donde calculo la desviación de un valor medio, no se porque le puse varianza
def varianza(df, columna, valor):
    df['varianza']=valor-df[columna]
    return df

## Audios

In [3]:
#Señales de audio
audio_file2 = "H:/Mi unidad/Análisis de audio/Audio 3/sonido3_2.mp3"
audio_file = "H:/Mi unidad/Análisis de audio/Audio 3/sonido3_1.mp3"

#Disminuyo la frecuencia de muestreo para que no explote la compu
tambor, fst = librosa.load(audio_file, sr=8000)
metronomo, fsm = librosa.load(audio_file2, sr=8000)
len(metronomo)

1451862

## Código

In [4]:
#.describe() de mis datos 
tambor_ampl = pd.Series(tambor)
df_tambor_amplitud=pd.DataFrame(tambor_ampl.describe())
metronomo_ampl = pd.Series(metronomo)
df_metronomo_amplitud=pd.DataFrame(metronomo_ampl.describe())
df_datos=pd.concat([df_tambor_amplitud, df_metronomo_amplitud], axis=1)
df_datos.columns = ['ampl-tambor', 'ampl-metronomo']

In [5]:
#Agrego una columna con el tiempo en segundos
dftambor=pd.DataFrame(tambor)
dftambor['tiempo_tambor'] = dftambor.index /fst
dftambor.columns = ['amplitud_tambor', 'tiempo_tambor']

dfmetronomo=pd.DataFrame(metronomo)
dfmetronomo['tiempo_metronomo'] = dfmetronomo.index /fsm
dfmetronomo.columns=['amplitud_metronomo', 'tiempo_metronomo']

In [6]:
#Corte de los dataframes en cuatro partes por los cambios de ritmo del metrónomo.
dftambor0=corte(45000, 320000, dftambor) 
dftambor1=corte(320000, 1400000, dftambor) 

dfmetronomo0=corte(45000, 320000, dfmetronomo)
dfmetronomo1=corte(320000, 1400000, dfmetronomo)


In [7]:
#Dataframe con los máximos
t_0=maximos(dftambor0, 'amplitud_tambor', 3000, fst, 'tiempo_tambor')
m_0=maximos(dfmetronomo0, 'amplitud_metronomo', 3000, fsm, 'tiempo_metronomo')
t_1=maximos(dftambor1, 'amplitud_tambor', 3000, fst, 'tiempo_tambor')
m_1=maximos(dfmetronomo1, 'amplitud_metronomo', 3000, fsm, 'tiempo_metronomo')



In [13]:
pd.options.display.max_rows = None
#Calculo la diferencia entre picos de cada parte
deltatambor0=delta(t_0, 'tiempo_tambor')
deltatambor0=varianza(deltatambor0, 'tiempo_tambor', deltametronomo0['tiempo_metronomo'].mean())
deltatambor1=delta(t_1, 'tiempo_tambor')
deltatambor1=varianza(deltatambor1, 'tiempo_tambor', deltametronomo1['tiempo_metronomo'].mean())

deltametronomo0=delta(m_0, 'tiempo_metronomo')
deltametronomo1=delta(m_1, 'tiempo_metronomo')

In [9]:
#Dataframes donde las amplitudes son 0 excepto en los máximos donde valen 1.
t_0g=maximos2(dftambor0, 'amplitud_tambor', 3000, fst, 'tiempo_tambor', 'indice')
m_0g=maximos2(dfmetronomo0, 'amplitud_metronomo', 3000, fsm, 'tiempo_metronomo', 'indice')
t_1g=maximos2(dftambor1, 'amplitud_tambor', 3000, fst, 'tiempo_tambor', 'indice')
m_1g=maximos2(dfmetronomo1, 'amplitud_metronomo', 3000, fsm, 'tiempo_metronomo', 'indice')

In [10]:
#Concateno los máximos de las cuatro partes del audio.
maximos_tambor=pd.concat([t_0, t_1], axis=0)
maximos_metronomo=pd.concat([m_0, m_1], axis=0)
maximos_tambor_norm=pd.concat([t_0g, t_1g], axis=0)
maximos_metronomo_norm=pd.concat([m_0g, m_1g], axis=0)

In [11]:
tiempos0, _ = procesar_picos(m_0, t_0)
tiempos1, _ = procesar_picos(m_1, t_1)

## Guardado de dataframes

In [14]:
#Guardo los dataframes.
dftambor.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/dftambor.csv', index=False)
dfmetronomo.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/dfmetronomo.csv', index=False)

dftambor0.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/dftambor0.csv', index=False)
dfmetronomo0.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/dfmetronomo0.csv', index=False)
dftambor1.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/dftambor1.csv', index=False)
dfmetronomo1.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/dfmetronomo1.csv', index=False)
                   
tiempos0.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/tiempos0.csv', index=False)
tiempos1.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/tiempos1.csv', index=False)

t_0.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/t_0.csv', index=False)
t_1.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/t_1.csv', index=False)
m_0.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/m_0.csv', index=False)
m_1.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/m_1.csv', index=False)

t_0g.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/t_0g.csv', index=False)
t_1g.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/t_1g.csv', index=False)
m_0g.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/m_0g.csv', index=False)
m_1g.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/m_1g.csv', index=False)

maximos_tambor.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/maximos_tambor.csv', index=False)
maximos_metronomo.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/maximos_metronomo.csv', index=False)
maximos_tambor_norm.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/maximos_tambor_norm.csv', index=False)
maximos_metronomo_norm.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/maximos_metronomo_norm.csv', index=False)

deltatambor0.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/deltatambor0.csv', index=False)
deltatambor1.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/deltatambor1.csv', index=False)
deltametronomo0.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/deltametronomo0.csv', index=False)
deltametronomo1.to_csv('H:/Mi unidad/Análisis de audio/Audio 3/deltametronomo1.csv', index=False)